<a href="https://colab.research.google.com/github/RS201918703/ST207-SQL-Project/blob/main/ST207_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ST207 Project - Amazon Notebook**

Please find attached the finalised CSV file of our dataset named **amazon.csv** with our submission. The original dataset before changes can be found here: https://app.datastock.shop/?site_name=Amazon.com_Product_Reviews (You will need a Datastock account to download the free version). We also added some additional data from https://www.kaggle.com/promptcloud/amazon-product-reviews-dataset. We made changes according to the Dataset section of our report.

_Please use the amazon.csv dataset to run the below code._
_This code was run using Google Collab_

In [ ]:
import pandas as pd
import sqlite3 as sql

In [ ]:
connection = sql.connect("/content/ST207_Project_Database.db")
c = connection.cursor()

In [ ]:
amazon = pd.read_csv("/content/amazon.csv")

In [ ]:
amazon.to_sql('amazon', con = connection, index = False) 

In [ ]:
drop_null_query ='''
DELETE FROM amazon
WHERE
  (UniqId IS NULL OR UniqId  = '');
'''

In [ ]:
c.execute(drop_null_query)

In [ ]:
#Creating the tables
create_customers_query = '''
CREATE TABLE IF NOT EXISTS Customers (
	UserId TEXT PRIMARY KEY,
	Location TEXT)
'''

create_products_query = '''CREATE TABLE IF NOT EXISTS Products (
	ProductID TEXT PRIMARY KEY,
  Category TEXT NOT NULL,
	SubCategory TEXT NOT NULL,
	Brand TEXT NOT NULL,
	ProdDescription TEXT NOT NULL)
'''

create_orders_query = '''CREATE TABLE IF NOT EXISTS Orders (
	BillingId TEXT PRIMARY KEY,
	ProdDescription TEXT NOT NULL,
	UserId TEXT NOT NULL,
	ProductId TEXT NOT NULL)
'''

create_reviews_query = '''CREATE TABLE IF NOT EXISTS Reviews (
	UniqId TEXT PRIMARY KEY,
	RTitle TEXT,
	RMonth TEXT,
	RRating TEXT,
	RContent TEXT,
	HelpfulR TEXT,
	UserId TEXT,
  BillingId TEXT,
  ProductId TEXT)
'''

The temporary tables are in place to clean up the data before we insert them into the main tables of our database.

In [ ]:
#Creating temporary tables
create_tempproducts_query = '''CREATE TABLE temp_products AS
SELECT DISTINCT
   ProductID,
   Category, 
   SubCategory,
   Brand,
   ProdDescription
FROM amazon
'''

delete_null_products_query = '''DELETE FROM temp_products
WHERE
  (Category IS NULL OR Category  = '')
  OR (Brand IS NULL OR Brand = '')
  OR (SubCategory IS NULL OR SubCategory = '')
  OR (ProdDescription IS NULL OR ProdDescription = '');
'''

create_temporders_query = '''CREATE TABLE temp_orders AS
SELECT DISTINCT
   BillingID,
   ProductID,
   UserId,
   ProdDescription
FROM amazon
'''

delete_null_orders_query = '''DELETE FROM temp_orders
WHERE
  (UserId IS NULL OR UserId  = '')
  OR (ProductId IS NULL OR ProductId = '')
  OR (ProdDescription IS NULL OR ProdDescription = '')
'''

create_tempreviews_query = '''CREATE TABLE IF NOT EXISTS temp_reviews (
	UniqId TEXT PRIMARY KEY,
	RTitle TEXT,
	RMonth TEXT,
	RRating TEXT,
	RContent TEXT,
	HelpfulR TEXT,
	UserId TEXT NOT NULL,
  BillingId TEXT NOT NULL,
  ProductId TEXT NOT NULL);
'''

insert_tempreviews_query = '''INSERT INTO temp_reviews(UniqId, RTitle, RMonth, RRating, RContent, HelpfulR, UserId, BillingId, ProductId)
SELECT UniqId, RTitle, RMonth, RRating, RContent, HelpfulR, UserId, BillingID, ProductID
FROM amazon;
'''

delete_null_reviews_query = '''
DELETE FROM temp_reviews
WHERE
  (UniqId IS NULL OR UniqId  = '')
'''



In [ ]:
c.execute(create_customers_query)
c.execute(create_products_query)
c.execute(create_orders_query)
c.execute(create_reviews_query)

c.execute(create_tempproducts_query)
c.execute(create_temporders_query)
c.execute(delete_null_orders_query)
c.execute(create_tempreviews_query)
c.execute(insert_tempreviews_query)
c.execute(delete_null_reviews_query)
c.execute(delete_null_products_query)
connection.commit()

The triggers for the database are created based on the restrictions we have discussed.

In [ ]:
#Triggers

#Trigger 1 - Ratings can only be in between values 1 and 5
trigger_1 = '''
CREATE TRIGGER bef_update_RRating BEFORE UPDATE ON Reviews
BEGIN
	SELECT CASE
	WHEN ((SELECT Reviews.RRating FROM Reviews WHERE NEW.RRating < 1 OR NEW.RRating > 5) IS NOT NULL)
	THEN RAISE(FAIL, 'ERROR: Invalid rating score.')
END;
END;
'''

#Trigger 2 - Ensuring each order can only have one review
trigger_2 = '''
CREATE TRIGGER bef_insert_review BEFORE INSERT ON Reviews
BEGIN
	SELECT CASE
	WHEN ((SELECT Reviews.BillingID FROM Reviews WHERE Reviews.BillingID = NEW.BillingID) IS NOT NULL)
	THEN RAISE(FAIL, 'ERROR: This order already has a review.')
END;
END;'''

#Trigger 3 - When inserting new foreign key value, the new keys must match a primary key
trigger_3_1 = '''CREATE TRIGGER bef_insert_orders_UserId BEFORE INSERT ON Orders
BEGIN
	SELECT CASE
	WHEN ((SELECT COUNT(UserId) FROM Customers WHERE Customers.UserId = NEW.UserId) = 0)
	THEN RAISE(FAIL, 'ERROR: Foreign key violation: This UserId does not have a matching record in the database.')
END;
END;'''

trigger_3_2 = '''CREATE TRIGGER bef_insert_orders_ProductId BEFORE INSERT ON Orders
BEGIN
	SELECT CASE
	WHEN ((SELECT COUNT(ProductId) FROM Products WHERE Products.ProductId = NEW.ProductId) = 0)
	THEN RAISE(FAIL, 'ERROR: Foreign key violation: This ProductId does not have a matching record in the database.')
END;
END;'''

trigger_3_3 = '''CREATE TRIGGER bef_insert_reviews_UserId BEFORE INSERT ON Reviews
BEGIN
	SELECT CASE
	WHEN ((SELECT COUNT(UserId) FROM Customers WHERE Customers.UserId = NEW.UserId) = 0)
	THEN RAISE(FAIL, 'ERROR: Foreign key violation: This UserId does not have a matching record in the database.')
END;
END;'''

trigger_3_4 = '''CREATE TRIGGER bef_insert_reviews_BillingId BEFORE INSERT ON Reviews
BEGIN
	SELECT CASE
	WHEN ((SELECT COUNT(BillingId) FROM Orders WHERE Orders.BillingId = NEW.BillingId) = 0)
	THEN RAISE(FAIL, 'ERROR: Foreign key violation: This BillingId does not have a matching record in the database.')
END;
END;'''

trigger_3_5 = '''CREATE TRIGGER bef_insert_reviews_ProductId BEFORE INSERT ON Reviews
BEGIN
	SELECT CASE
	WHEN ((SELECT COUNT(ProductId) FROM Products WHERE Products.ProductId = NEW.ProductId) = 0)
	THEN RAISE(FAIL, 'ERROR: Foreign key violation: This ProductId does not have a matching record in the database.')
END;
END;
'''

#Trigger 4 - No records are allowed to be deleted
trigger_4_1 = '''
CREATE TRIGGER bef_delete_1 BEFORE DELETE ON Customers
BEGIN
	SELECT RAISE(ABORT, 'ERROR: Records cannot be deleted.');
END;'''

trigger_4_2 = '''CREATE TRIGGER bef_delete_2 BEFORE DELETE ON Orders
BEGIN
	SELECT RAISE(ABORT, 'ERROR: Records cannot be deleted.');
END;'''

trigger_4_3 = '''CREATE TRIGGER bef_delete_3 BEFORE DELETE ON Reviews
BEGIN
	SELECT RAISE(ABORT, 'ERROR: Records cannot be deleted.');
END;'''

trigger_4_4 = '''CREATE TRIGGER bef_delete_4 BEFORE DELETE ON Products
BEGIN
	SELECT RAISE(ABORT, 'ERROR: Records cannot be deleted.');
END;
'''

In [ ]:
c.execute(trigger_1)
c.execute(trigger_2)
c.execute(trigger_3_1)
c.execute(trigger_3_2)
c.execute(trigger_3_3)
c.execute(trigger_3_4)
c.execute(trigger_3_5)
c.execute(trigger_4_1)
c.execute(trigger_4_2)
c.execute(trigger_4_3)
c.execute(trigger_4_4)

connection.commit()

In [ ]:
#Inserting data
insert_customers_query = '''INSERT INTO Customers (UserId, Location)
SELECT UserId, Location
FROM amazon
'''
delete_null_customers_query = '''
DELETE FROM Customers
WHERE
  (UserId IS NULL OR UserId  = '')
'''

insert_products_query = '''INSERT INTO Products(ProductId, Category, SubCategory, Brand, ProdDescription)
SELECT    ProductId, Category, SubCategory, Brand, ProdDescription
FROM temp_products
'''

insert_orders_query = '''INSERT INTO Orders(BillingId, ProdDescription, UserId, ProductId)
SELECT BillingId, ProdDescription, UserId, ProductId
FROM temp_orders
'''

insert_reviews_query = '''INSERT INTO Reviews(UniqId, RTitle, RMonth, RRating,RContent,HelpfulR,UserId,BillingId,ProductId)
SELECT temp_reviews.UniqId, temp_reviews.RTitle, temp_reviews.RMonth, temp_reviews.RRating, temp_reviews.RContent, temp_reviews.HelpfulR, temp_reviews.UserId, Orders.BillingId, Orders.ProductId 
FROM temp_reviews
JOIN Orders ON temp_reviews.UserId = Orders.UserId;
'''


drop_tempproducts_query = '''DROP TABLE temp_products;'''
drop_temporders_query = '''DROP TABLE temp_orders;'''
drop_tempreviews_query = '''DROP TABLE temp_reviews;'''
drop_maindatatable_query = '''DROP TABLE amazon;'''

In [ ]:
c.execute(insert_customers_query)
c.execute(delete_null_customers_query)
c.execute(insert_products_query)
c.execute(insert_orders_query)
c.execute(insert_reviews_query)
c.execute(delete_null_products_query)

c.execute(drop_tempproducts_query)
c.execute(drop_temporders_query)
c.execute(drop_tempreviews_query)
c.execute(drop_maindatatable_query)
connection.commit()

The following code is used to test the triggers in our database. All of them should raise errors.

In [ ]:
#Testing trigger 1 (Lower bound) - Error should raise
c.execute('''
UPDATE Reviews
   SET RRating = -5
 WHERE UniqID = "1627ee2055d36db00459430b5ec3790d";
''')

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



IntegrityError: ignored

In [ ]:
#Testing trigger 1 (Upper bound) - Error should raise
c.execute('''
UPDATE Reviews
   SET RRating = 10
 WHERE UniqID = "1627ee2055d36db00459430b5ec3790d";
''')

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



IntegrityError: ignored

In [ ]:
#Testing trigger 2 - Inserting another review for an order with an existing review should raise an error 
c.execute('''INSERT INTO Reviews
VALUES ('9999ee2055d36db00459430b5ec3790dnew', 'None', 'July', 5.0, 'Like it', 0, 'AECRD4GDEG7RBMTQDOXCEN6NT74Q', '141', '1');
''') 

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



IntegrityError: ignored

In [ ]:
#Testing trigger 3 - Inserting a UserId which does not exist as a PK elsewhere in the Orders table - Error should raise
c.execute('''
INSERT INTO Orders
VALUES ('9000000000000','Desc','TestID','1');
''')

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



IntegrityError: ignored

In [ ]:
#Testing trigger 3 - Inserting a ProductId which does not exist as a PK elsewhere in the Orders table - Error should raise
c.execute('''
INSERT INTO Orders
VALUES ('9000000000000','Desc','AFUNJ4TSQOUYJ7CYJ24MYKCH3QDQ','TestID');
''')

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



IntegrityError: ignored

In [ ]:
#Testing trigger 3 - Inserting a UserId which does not exist as a PK elsewhere in the Reviews table - Error should raise
c.execute('''INSERT INTO Reviews
VALUES ('New test','Title','June',4.0,'OK','5','TestID','141','1');
''')

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



IntegrityError: ignored

In [ ]:
#Testing trigger 3 - Inserting a BillingId which does not exist as a PK elsewhere in the Reviews table - Error should raise
c.execute('''
INSERT INTO Reviews
VALUES ('New test','Title','June',4.0,'OK','5','AECRD4GDEG7RBMTQDOXCEN6NT74Q','TestID','1');
''')

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



IntegrityError: ignored

In [ ]:
#Testing trigger 3 - Inserting a ProductId which does not exist as a PK elsewhere in the Reviews table - Error should raise
c.execute('''
INSERT INTO Reviews
VALUES ('New test','Title','June',4.0,'OK','5','AECRD4GDEG7RBMTQDOXCEN6NT74Q','142','TestID');
''')

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



IntegrityError: ignored

In [ ]:
#Testing trigger 4 - Error should raise
c.execute('''
DELETE FROM Customers;
''')

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



IntegrityError: ignored

In [ ]:
#Testing for primary key restrictions - Customers table
c.execute('''INSERT INTO Customers
VALUES ('AECRD4GDEG7RBMTQDOXCEN6NT74Q','Brazil');
''')

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



IntegrityError: ignored

In [ ]:
#Testing for primary key restrictions - Orders table
c.execute('''INSERT INTO Orders
VALUES (141,'This massive volume shampoo is tigi''s most advance volume shampoo ever. The shampoo releases its innovative technology to give volume you can see and fell.','AEDDFG2AFT4GRGAG2KCYXAAMWZ7A','1');
''')

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



IntegrityError: ignored

In [ ]:
#Testing for primary key restrictions - Reviews table - Error should raise
c.execute('''
INSERT INTO Reviews
VALUES ('1627ee2055d36db00459430b5ec3790d','Test','June','4.0','OK','0','AECRD4GDEG7RBMTQDOXCEN6NT74Q','Test','SCS');
''')

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



IntegrityError: ignored

In [ ]:
#Testing for primary key restrictions - Products table - Error should raise
c.execute('''
INSERT INTO Products
VALUES ('1','Beauty & Personal Care','Hair Care','New brand','Description');
''')

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



IntegrityError: ignored

Index 1 is used to search through the Helpful Reviews column in the Reviews table. This index helps the database in searching for reviews that other customers find helpful while also gaining an understanding of the general customer sentiment towards specific products.

In [ ]:
#Index on Helpful Reviews 
c.execute('''
CREATE INDEX index_helpful
ON Reviews(HelpfulR);
''')

Index 2 is used to search through the Locations column in the Customers table. This can be particularly useful for database users if they would like to do an analysis of how Amazon is performing in certain areas of the world. Are certain products more popular in a certain country? Is business particularly performing well in a different country? These are all questions that can be answered by making queries using the location index. 

In [ ]:
#Index on Customer Locations
c.execute('''
CREATE INDEX index_locations
ON Customers(Location);
''')

Index 3 is a unique index that is made using the Billing ID column of the Orders table. This can be used to search through several orders to find information that may be relevant to database users. An example would be searching for particular products that are common in a large number of orders. 

In [ ]:
#Index on Order Billing IDs
c.execute('''
CREATE UNIQUE INDEX index_billing_id
ON Orders(BillingId);
''')

Index 4 is a composite index made on the category and subcategory columns in the Product table. The primary use of this index is searching across different categories and analysing which products in certain categories are performing the best in terms of both orders and positive reviews. 

In [ ]:
#Index on Categories and Sub-Categories
c.execute('''
CREATE INDEX index_cat_subcat
ON Products(Category, SubCategory);
''')

In [ ]:
index_plan1 = c.execute('''
EXPLAIN QUERY PLAN
SELECT *
FROM Reviews
WHERE HelpfulR > 0;
''').fetchall()

pd.DataFrame(index_plan1)

,0,1,2,3
0,0,0,0,SEARCH TABLE Reviews USING INDEX index_helpful...


In [ ]:
index_plan2 = c.execute('''
EXPLAIN QUERY PLAN
SELECT ProductId
FROM Orders
WHERE BillingId > 100;
''').fetchall()

pd.DataFrame(index_plan2)

,0,1,2,3
0,0,0,0,SEARCH TABLE Orders USING INDEX index_billing_...


In [ ]:
index_plan3 = c.execute('''
EXPLAIN QUERY PLAN
SELECT Location, Count(*)
FROM Customers 
GROUP BY Location
ORDER BY Count(*)DESC
''').fetchall()

pd.DataFrame(index_plan3)

,0,1,2,3
0,0,0,0,SCAN TABLE Customers USING COVERING INDEX inde...
1,0,0,0,USE TEMP B-TREE FOR ORDER BY


In [ ]:
index_plan4 = c.execute('''
EXPLAIN QUERY PLAN
SELECT ProductId
FROM Products
WHERE Category = 'Beauty & Personal Care' AND SubCategory = 'Frozen';
''').fetchall()

pd.DataFrame(index_plan4)

,0,1,2,3
0,0,0,0,SEARCH TABLE Products USING INDEX index_cat_su...


In [ ]:
index_list = c.execute('''
SELECT type, name, tbl_name, sql
FROM   sqlite_master
WHERE  type = 'index';
''').fetchall()

pd.DataFrame(index_list)

,0,1,2,3
0,index,sqlite_autoindex_Customers_1,Customers,None
1,index,sqlite_autoindex_Products_1,Products,None
2,index,sqlite_autoindex_Orders_1,Orders,None
3,index,sqlite_autoindex_Reviews_1,Reviews,None
4,index,index_helpful,Reviews,CREATE INDEX index_helpful\nON Reviews(HelpfulR)
5,index,index_locations,Customers,CREATE INDEX index_locations\nON Customers(Loc...
6,index,index_billing_id,Orders,CREATE UNIQUE INDEX index_billing_id\nON Order...
7,index,index_cat_subcat,Products,CREATE INDEX index_cat_subcat\nON Products(Cat...


View 1 contains review content, as well as the location of the customers who other users have determined, have left a helpful review.

In [ ]:
#View 1
c.execute('''
CREATE VIEW helpful_reviews AS
SELECT Reviews.RTitle, Reviews.RContent, Reviews.HelpfulR, Customers.Location
FROM Reviews
JOIN Customers
ON Reviews.UserId = Customers.UserId
WHERE HelpfulR != 0;
''')

View 2 has review and product information on products that had a review of 4 or 5 out of 5. This makes queries concerning only the highest rated products and identifying anomalously high reviews, easier.


In [ ]:
#View 2
c.execute('''
CREATE VIEW highly_rated_products AS
SELECT Products.Brand, Products.ProdDescription, Reviews.RRating, Reviews.RTitle, Reviews.RContent
FROM Reviews
LEFT JOIN Products
ON Reviews.ProductId = Products.ProductId
WHERE Reviews.RRating > 3.0;
''')

Query 1: Let’s say that we have a customer who is looking at a particular product with the ProductId of ‘153’. They are unsure whether they want to buy this product so they send a query to retrieve all the reviews of the product. The query will rank reviews by how helpful they are using HelpfulR.

In [ ]:
#Query 1
Query1 = c.execute('''
SELECT RRating, RTitle, RContent, HelpfulR
FROM Reviews
JOIN Products ON Products.ProductId=Reviews.ProductId
WHERE Reviews.ProductId = 153
ORDER BY HelpfulR DESC;
''').fetchall()
pd.DataFrame(Query1,columns=['Review rating', 'Review title','Review content','Helpful review'])

,Review rating,Review title,Review content,Helpful review
0,5,Shh it is a secret!,To the uninitiated (someone who didn't grow up...,4
1,5,"This shit is like 80% MSG, but it tastes great!","This stuff tastes amazing, but it�۪ll probably...",4
2,3,not very strong flavor.,not very strong flavor... need to use more tha...,1
3,5,Very nice and quick shipping,Very nice and quick shipping. This is the powd...,0
4,4,ordeered it for friends,I personally do not use this product. I ordere...,0
5,5,Excellent,"Excellent Product for any food , Flavor",0
6,4,salty,salty,0
7,5,Would recommend,Great stuff i use it for soups and flavors for...,0
8,5,Enhances flavor.,Great addition for homemade soups and gravies.,0
9,5,Goood taste!,My dad loves it!,0


Query 2: Similar to query 1, the idea behind this query is to act as a guide for a customer. If a customer is unsure what product they would like (e.g. for a gift), they can search through products that have had reviews with a generally positive sentiment

In [ ]:
#Query 2
Query2 = c.execute('''
SELECT Products.Brand, Products.ProdDescription, Reviews.RRating, Reviews.RTitle, Reviews.RContent
FROM Reviews
LEFT JOIN Products
ON Reviews.ProductId = Products.ProductId
WHERE Reviews.RTitle LIKE '%good%' OR  Reviews.RTitle LIKE '%great%' OR Reviews.RContent LIKE '%good%' OR Reviews.RContent LIKE '%great%'
''').fetchall()
pd.DataFrame(Query2,columns=['Brand','Product Description','Review Rating','Review Title',' Review Content'])

,Brand,Product Description,Review Rating,Review Title,Review Content
0,Suave,Bring 73othness and shine back to damaged hair...,2,"Shampoo not a good hair product, used it for y...",I ordered the shampoo and conditioner over Ama...
1,Suave,Bring 73othness and shine back to damaged hair...,5,Good for thinning hair,Awesome product
2,Suave,Bring 73othness and shine back to damaged hair...,5,It's good for my hair,I like the smell and the way it leaves my hair
3,Suave,Bring 73othness and shine back to damaged hair...,5,Good,Good
4,Suave,Bring 73othness and shine back to damaged hair...,5,Five Stars,Good 2 pack. Items were in good condition
...,...,...,...,...,...
71,SheaMoisture,Gently cleanse and hydrate aging skin with thi...,5,This is my favorite soap,I use this soap for my sensitive skin. It�s mo...
72,SheaMoisture,Gently cleanse and hydrate aging skin with thi...,5,Good Product,I bought this soap for my daughter and daughte...
73,SheaMoisture,Gently cleanse and hydrate aging skin with thi...,5,Great value,Really like this soap so far. Been using it fo...
74,Talenti,Our Salted Caramel Truffle is an ode to our be...,5,Heavenly __,So delicious. I can only buy this every few mo...


Query 3: This query is an extension of the previous query where customers can view the most highly rated products from amongst a particular subcategory - in this case Hair Care.


In [ ]:
#Query 3
Query3 = c.execute('''
SELECT Products.ProductId, Avg(Reviews.RRating)
FROM Products
JOIN Reviews ON Reviews.ProductId = Products.ProductID
WHERE Products.ProductId IN
(SELECT ProductId FROM Products WHERE Products.SubCategory = 'Hair Care')
GROUP BY Products.ProductId
ORDER BY Avg(RRating) DESC
''').fetchall()
pd.DataFrame(Query3,columns=['ProductId','Average rating'])

,ProductId,Average rating
0,22,4.900000
1,32,4.888889
2,1,4.800000
3,134,4.700000
4,144,4.444444
5,112,4.111111


Query 4: This query retrieves the average review for each brand across the dataset. This has an application for both customers and producers, as customers can see the average quality of a product they might be about to buy, and producers can see which of their competitors is most highly favoured at any given time

In [ ]:
#Query 4
Query4 = c.execute('''
SELECT Products.Brand, round(avg(Reviews.RRating), 1) As 'Average Rating'
FROM Reviews
LEFT JOIN Products
ON Reviews.ProductId = Products.ProductId
GROUP BY Products.Brand
ORDER BY round(avg(Reviews.RRating), 1) DESC
''').fetchall()
pd.DataFrame(Query4,columns=['Brand','Average rating'])

,Brand,Average rating
0,Clinique,5.0
1,DOVE WOMENS DEO,5.0
2,PG Tips,5.0
3,St. Ives,5.0
4,Sir Kensingtons,4.8
5,Suave,4.8
6,TIGI,4.7
7,SheaMoisture,4.6
8,Bed Head,4.4
9,AXE,4.3


Query 5: This query retrieves useful information for companies. It counts the number of orders for each brand. It will give companies a good idea of who has the largest market share

In [ ]:
#Query 5
Query5 = c.execute('''
SELECT Products.Brand, count(*)
FROM Orders
LEFT JOIN Products
ON Orders.ProdDescription = Products.ProdDescription
GROUP BY Products.Brand
''').fetchall()
pd.DataFrame(Query5,columns=['Brand','Order total'])

,Brand,Order total
0,AXE,9
1,Bed Head,14
2,Clinique,1
3,DOVE WOMENS DEO,1
4,Knorr,13
5,Lipton,10
6,PG Tips,10
7,Pure Leaf,3
8,Schmidt's Deodorant,17
9,SheaMoisture,14


Query 6: This query counts the number of orders for each product grouped by country. It will be useful as it gives insights to producers on where their products are selling the best.

In [ ]:
#Query 6
Query6 = c.execute('''
SELECT Orders.ProductId, Customers.Location, Count(*) AS Number_Of_Purchases
FROM Orders, Customers
WHERE Orders.UserId = Customers.UserId
GROUP BY Customers.Location, Orders.ProductId
ORDER BY Number_Of_Purchases DESC
''').fetchall()
pd.DataFrame(Query6,columns=['ProductId','Location','Order total'])

,ProductId,Location,Order total
0,173,Germany,5
1,22,UK,4
2,134,USA,4
3,46,Australia,3
4,102,Ghana,3
...,...,...,...
127,16,USA,1
128,22,USA,1
129,42,USA,1
130,56,USA,1


Query 7: This query calculates the number of reviews by month. This is a producer application as this query gives us an insight into when customers buy products.

In [ ]:
#Query 7
Query7 = c.execute('''
SELECT Reviews.RMonth, count(*)
FROM Orders
LEFT JOIN Reviews
ON Orders.BillingId = Reviews.BillingId
GROUP BY Reviews.RMonth
ORDER BY case when RMonth = 'January' then 1
              when RMonth = 'February' then 2
              when RMonth = 'March' then 3
              when RMonth = 'April' then 4
              when RMonth = 'May' then 5
              when RMonth = 'June' then 6
              when RMonth = 'July' then 7
              when RMonth = 'August' then 8
              when RMonth = 'September' then 9
              when RMonth = 'October' then 10
              when RMonth = 'November' then 11
              else 12
         end;
''').fetchall()
pd.DataFrame(Query7,columns=['Month','Review total'])

,Month,Review total
0,January,50
1,February,21
2,March,16
3,April,5
4,May,4
5,June,9
6,July,14
7,August,8
8,September,11
9,October,14


Query 8: This gives a viewpoint from a business standpoint showing the top 3 countries where Skin Care products are purchased on Amazon.


In [ ]:
Query8 = c.execute('''
SELECT Customers.Location, Count(*) AS Number_Of_Purchases
FROM Customers
JOIN Orders ON Orders.UserId = Customers.UserId
WHERE Orders.ProductId IN
(SELECT Orders.ProductID FROM Orders WHERE Orders.ProductId IN
(SELECT Products.ProductId FROM Products WHERE Products.SubCategory = 'Skin Care'))
GROUP BY Customers.Location
ORDER BY Number_Of_Purchases DESC
LIMIT 3
''').fetchall()

pd.DataFrame(Query8,columns=['Location','Skin care orders'])

,Location,Skin care orders
0,Australia,5
1,France,4
2,Italy,4


Query 9: This shows an update query where we update the location of a particular customer who may have moved countries. We then use a query to showcase the new location.


In [ ]:
Update1 = c.execute('''
UPDATE Customers
SET Location = 'Canada'
WHERE UserId = 'AFPOC3H4CUZWZUI3D44H25MBPYEQ'
''')
Query9 = c.execute('''
SELECT *
FROM Customers
WHERE Location = 'Canada'
''').fetchall()

pd.DataFrame(Query9,columns=['UserId','Location'])

,UserId,Location
0,AFPOC3H4CUZWZUI3D44H25MBPYEQ,Canada


Query 10: Here we have an update query which is designed to work for companies on Amazon. If producers want to make changes to their products, it should be reflected in the database with this query.

In [ ]:
Update2 = c.execute('''
UPDATE Products
SET ProdDescription = '36 Gallon Size Lipton Tropical Iced Tea Bags (1 Box per order) by Lipton. Specially blended for iced tea. Fresh brew.'
WHERE ProductId = 173
''')

Query10 = c.execute('''
SELECT ProdDescription
FROM Products
WHERE ProductId = 173
''').fetchall()

pd.DataFrame(Query10,columns=['Product description'])

,Product description
0,36 Gallon Size Lipton Tropical Iced Tea Bags (...
